# Exploratory Data Analysis for Olist Ecommerce Dataset
This notebook will explore the Olist Ecommerce Dataset to identify key characteristics useful for predicting customer churn.

First, import libraries and configure dataset directory.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

DATA_DIR = 'archive'

Load each dataset as a pandas DataFrame.

In [ ]:
print("Loading datasets...")

try:
    customers_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_customers_dataset.csv'))
    orders_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))
    order_items_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
    order_payments_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_payments_dataset.csv'))
    # product_info needs joining orders->items->products later
    products_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))
    # reviews might be useful for features later
    reviews_df = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_reviews_dataset.csv'))

    print("Datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")
    print(f"Please ensure the CSV files are in the '{DATA_DIR}' directory.")

Get a first look at each dataframe.

In [ ]:
dataframes = {
    "Customers": customers_df,
    "Orders": orders_df,
    "Order Items": order_items_df,
    "Order Payments": order_payments_df,
    "Products": products_df,
    "Reviews": reviews_df
}

for name, df in dataframes.items():
    print(f"\n--- Inspecting: {name} DataFrame ---")
    print(f"Shape: {df.shape}") # (rows, columns)

    print("\nFirst 5 rows:")
    # Display more columns if needed: pd.set_option('display.max_columns', None)
    print(df.head())

    print("\nInfo (Data Types & Non-Null Counts):")
    # This is crucial for spotting missing values and wrong data types
    df.info()

    # Get basic statistics for numerical columns, only if they exist
    #print("\nDescriptive Statistics (Numerical Columns):")
    #numerical_cols = df.select_dtypes(include=np.number).columns
    #if not numerical_cols.empty:
    #    print(df.describe(include=[np.number])) # Use include=[np.number] to only show numerical stats initially
    #else:
    #    print("No numerical columns found in this DataFrame.")


    # Optional: Look at categorical descriptions if needed later
    # print("\nDescriptive Statistics (Categorical Columns):")
    # print(df.describe(include=['object']))

    print("-" * 50)

### Most Important Features Identified
- #### Recency: Days since customers last purchase
    - **Reasoning:**  Customers who haven't bought in a long time are likely to have churned
    - **Extraction:** Max of order_purchase_timestamp
- #### Frequency: Total number of orders a customer has placed
    - **Reasoning:** Customers who buy often are more engaged, less likely to churn
    - **Extraction:** Count unique order_ids associated with customer
- #### Tenure: Days between customers first and last purchase
    - **Reasoning:** Helps identify low-risk, loyal customers that are less likely to churn
    - **Extraction:** Difference of last purchase and first purchase in days 
- #### Monetary: Total amount of money customer has spent
    - **Reasoning:** High-spenders are valuable, and their spending habit is a strong engagement signal
    - **Extraction:** Sum of all order payment values associated with customer
- #### Average Payment Installments
    - **Reasoning:** Customers who pay in many installments likely place high value orders and experience different churn patterns
    - **Extraction:** Average of all payment installment amounts for orders associated with customer
- #### Preferred Payment Method
    - **Reasoning:** Customers may have different curn patterns based on preferred payment type
    - **Extraction:** Mode of all payment_type associated with customer
- #### Average Payment Complexity: Unique payment methods per single order
    - **Reasoning:** Customers who use multiple payment methods per order likely wait for vouchers and have a different churn profile
    - **Extraction:** Average of the max payment_sequential for each order associated with a customer
- #### Average Review Score
    - **Reasoning:** Customers who leave high reviews are more satisfied, less likely to churn
    - **Extraction:** Average of all reviews associated with customer
- #### Review Engagement Rate: How often does customer leave review messages
    - **Reasoning:** A customer who leaves review messages, good or bad, is more engaged and less likely to churn
    - **Extraction:** The ratio of orders with messages vs orders without messages for orders associated wtih customer
- #### Average Shipping Cost
    - **Reasoning:** Customers who pay more for shipping are more likely to churn
    - **Extraction:** Take the average sum of freight_value for every order item for orders associated with customer
- #### Number of Unique Product Categories Purchased
    - **Reasoning:** Customers who have bought products from more categories are more invested in the platform, less likely to churn
    - **Extraction:** Number of unique product categories associated with customer orders
- #### Number of Unique Sellers Purchased From
    - **Reasoning:** Customers who purchase form many sellers are more engaged, less likely to churn
    - **Extraction:** Total of unique seller_id for orders items associated with customer
- #### Average Payment Approval Time: Gap between order placement and approval
    - **Reasoning:** Customers who frequently experience delays between placing orders and order approval are more likely to churn
    - **Extraction:** Average difference between order approval and order placement
- #### Average Delivery vs Estimate
    - **Reasoning:** Customers who consistently get late packages are likely to churn
    - **Extraction:** Average time between order_delivered_customer_date and order_estimated_delivery_date
- #### Average Carrier Transit Time
    - **Reasoning:** Customers who experience long shipping times are more likely to churn
    - **Extraction:** Average difference between carier delivery and customer delivery of all orders associated with a customer
- #### Average Time Between Seller Shipping Deadline and Carrier Delivery
    - **Reasoning:** Sellers not meeting their fulfillment deadlines will make customers more liekly to churn
    - **Extraction:** Average difference between seller deadline and carrier delivery for orders associated with customer

### Data Cleaning
- The previous output shows that dates are stored as strings, they will need to be converted to datetime objects. 
- There are some missing orders that will need filtered out. 
- Products with missing categories will need to have their categories set to 'unknown'. 
- Can remove customer_zip_code_prefix, customer_city, customer_state, order_item_id, review_id, review_comment_title, all product columns except product_id and product_category.


In [ ]:
# --- 1. Clean and Minimize Orders DataFrame ---
try:
    print(f"Original shape: {orders_df.shape}")

    # Investigate 'order_status'
    print("\n'order_status' counts:")
    print(orders_df['order_status'].value_counts(dropna=False))

    # Filter for 'delivered' orders
    delivered_orders_df = orders_df[orders_df['order_status'] == 'delivered'].copy()
    print(f"\nFiltered for 'delivered' status. New shape: {delivered_orders_df.shape}")

    # Convert Timestamps
    print("\nConverting timestamp columns...")
    timestamp_cols = [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date'
    ]
    for col in timestamp_cols:
        delivered_orders_df[col] = pd.to_datetime(delivered_orders_df[col])
    
    # --- Create a Minimal DataFrame for Merging ---
    cleaned_orders_minimal = delivered_orders_df[['order_id','customer_id','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']]

    # Verify changes
    print("\nVerifying 'cleaned_orders_minimal' Dtypes:")
    cleaned_orders_minimal.info()

    # Save the cleaned file
    output_file_orders = f'{DATA_DIR}/cleaned_orders_minimal.csv'
    cleaned_orders_minimal.to_csv(output_file_orders, index=False)
    print(f"\nSuccessfully saved cleaned minimal orders data to: {output_file_orders}")
    dataframes["Orders"] = cleaned_orders_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")


# --- 2. Clean and Minimize Reviews DataFrame ---
print("\n--- Processing: Reviews DataFrame ---")
try:
    print(f"Original shape: {reviews_df.shape}")
    print("Converting timestamp columns...")

    review_ts_cols = ['review_creation_date', 'review_answer_timestamp']
    for col in review_ts_cols:
        reviews_df[col] = pd.to_datetime(reviews_df[col])

    cleaned_reviews_minimal = reviews_df[['order_id','review_score', 'review_comment_message']]
    
    # Verify changes
    print("\nVerifying 'cleaned_reviewes_minimal' Dtypes:")
    cleaned_reviews_minimal.info()

    # Save the cleaned file
    output_file_reviews = f'{DATA_DIR}/cleaned_reviews_minimal.csv'
    cleaned_reviews_minimal.to_csv(output_file_reviews, index=False)
    print(f"\nSuccessfully saved cleaned minimal reviews data to: {output_file_reviews}")
    dataframes["Reviews"] = cleaned_reviews_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")


# --- 3. Clean and Minimize Order Items DataFrame ---
print("\n--- Processing: Order Items DataFrame ---")
try:
    print(f"Original shape: {order_items_df.shape}")
    print("Converting 'shipping_limit_date' to datetime object...")
    
    order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

    cleaned_order_items_minimal = order_items_df[['order_id','product_id','seller_id','shipping_limit_date','price','freight_value']]
    
    # Verify changes
    print("\nVerifying 'cleaned_order_items_minimal' Dtypes:")
    cleaned_order_items_minimal.info()

    # Save the cleaned file
    output_file_items = f'{DATA_DIR}/cleaned_order_items_minimal.csv'
    cleaned_order_items_minimal.to_csv(output_file_items, index=False)
    print(f"\nSuccessfully saved cleaned minimal order items data to: {output_file_items}")
    dataframes["Order Items"] = cleaned_order_items_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 4. Clean and Minimize Products DataFrame ---
try:
    print(f"Original shape: {products_df.shape}")

    # --- 1. Inspect Missing Values (Before) ---
    missing_count = products_df['product_category_name'].isnull().sum()
    print(f"\nMissing 'product_category_name' values (Before): {missing_count}")

    # --- 2. Clean the Column ---
    # Fill NaN values with the string 'unknown'
    products_df['product_category_name'] = products_df['product_category_name'].fillna('unknown')
    print("Filled NaN values with 'unknown'.")

    # --- 3. Verify the Cleaning (After) ---
    missing_count_after = products_df['product_category_name'].isnull().sum()
    print(f"Missing 'product_category_name' values (After): {missing_count_after}")

    # --- 4. Create a Minimal DataFrame for Merging ---
    cleaned_products_minimal = products_df[['product_id', 'product_category_name']]
    
    print("\nHead of the cleaned, minimal products DataFrame:")
    print(cleaned_products_minimal.head())

    # --- 5. Save the Cleaned File ---
    output_file = f'{DATA_DIR}/cleaned_products_minimal.csv'
    cleaned_products_minimal.to_csv(output_file, index=False)
    print(f"\nSuccessfully saved cleaned minimal product data to: {output_file}")
    dataframes["Products"] = cleaned_products_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 5. Minimize Customer DataFrame ---
try:
    print(f"Original shape: {customers_df.shape}")

    # --- 1. Create a Minimal DataFrame for Merging ---
    customers_minimal = customers_df[['customer_id', 'customer_unique_id']]
    
    print("\nHead of the minimal customer DataFrame:")
    print(customers_minimal.head())

    # --- 2. Save the Minimal File ---
    output_file = f'{DATA_DIR}/customers_minimal.csv'
    customers_minimal.to_csv(output_file, index=False)
    print(f"\nSuccessfully saved minimal customer data to: {output_file}")
    dataframes["Customers"] = customers_minimal  # Update the reference

except Exception as e:
    print(f"\nAn error occurred: {e}")

print("\n--- All cleaning tasks complete. ---")

### Feature Engineering
Now that the dataframes are cleaned and minimized, it is time to calculate features and assemble the final customer centric dataframe. We'll start by calculating the recency, frequency, and tenure features.

In [ ]:
# Join customers and orders to link customer_unique_id to order_id
dataframes["Customer Orders"] = pd.merge(
    dataframes["Customers"],  # Has customer_id, customer_unique_id
    dataframes["Orders"],     # Has customer_id, order_id, and all dates
    on='customer_id'
)

# We need a "snapshot date" to calculate recency (1 day after the last purchase)
snapshot_date = dataframes["Customer Orders"]['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

# Group by customer and aggregate
rft_features = dataframes["Customer Orders"].groupby('customer_unique_id').agg(
    last_purchase=('order_purchase_timestamp', 'max'),
    Frequency=('order_id', 'nunique'),
    first_purchase=('order_purchase_timestamp', 'min')
).reset_index()

# Calculate Recency and Tenure in days
rft_features['Recency'] = (snapshot_date - rft_features['last_purchase']).dt.days
rft_features['Tenure'] = (rft_features['last_purchase'] - rft_features['first_purchase']).dt.days

# This is our starting point for the final DataFrame
dataframes["Customer Centric"] = rft_features[['customer_unique_id', 'Recency', 'Frequency', 'Tenure']]

Now moving on to the monetary and payment features...

In [ ]:
# Link payments to the customer/order master
dataframes["Merged Payments"] = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    dataframes["Order Payments"],
    on='order_id'
)

# 1. Aggregate basic payment features
payment_features = dataframes["Merged Payments"].groupby('customer_unique_id').agg(
    Monetary=('payment_value', 'sum'),
    avg_payment_installments=('payment_installments', 'mean'),
    preferred_payment_method=('payment_type', lambda x: x.mode()[0]) # Get the most frequent
).reset_index()

# 2. Calculate Payment Complexity (multi-step)
# Find the max sequential number for *each order*
order_complexity = dataframes["Merged Payments"].groupby(['customer_unique_id', 'order_id']) \
                                  .agg(max_sequential=('payment_sequential', 'max')) \
                                  .reset_index()
# Now, find the average complexity *per customer*
customer_complexity = order_complexity.groupby('customer_unique_id') \
                                      .agg(avg_payment_complexity=('max_sequential', 'mean')) \
                                      .reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], payment_features, on='customer_unique_id', how='left')
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], customer_complexity, on='customer_unique_id', how='left')

Next review features...

In [ ]:
reviews_merged = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    reviews_df,
    on='order_id'
)

# Aggregate review features
review_features = reviews_merged.groupby('customer_unique_id').agg(
    avg_review_score=('review_score', 'mean'),
    # .mean() on a boolean (notnull()) gives the ratio/rate
    review_engagement_rate=('review_comment_message', lambda x: x.notnull().mean()) 
).reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], review_features, on='customer_unique_id', how='left')

Item, product, and seller features...

In [ ]:
# Link items to customers/orders
items_merged = pd.merge(
    dataframes["Customer Orders"][['customer_unique_id', 'order_id']],
    dataframes["Order Items"],
    on='order_id'
)
# Now link to products to get the category
products_merged = pd.merge(
    items_merged,
    products_df,
    on='product_id'
)

# Aggregate features from this combined table
item_features = products_merged.groupby('customer_unique_id').agg(
    total_freight_value=('freight_value', 'sum'), # We'll calculate avg cost later
    unique_product_categories=('product_category_name', 'nunique'),
    unique_sellers=('seller_id', 'nunique')
).reset_index()

# --- Merge this group into the main DataFrame ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], item_features, on='customer_unique_id', how='left')

# Create the Average Shipping Cost (using total_freight and Frequency)
dataframes["Customer Centric"]['avg_shipping_cost'] = (
    dataframes["Customer Centric"].pop('total_freight_value') / dataframes["Customer Centric"]['Frequency']
)

Gap time features...

In [ ]:
# We need to link items to get the shipping_limit_date for each order
gaps_merged = pd.merge(
    dataframes["Customer Orders"],
    dataframes["Order Items"][['order_id', 'shipping_limit_date']],
    on='order_id'
)

# De-duplicate: An order with 3 items will have 3 rows. We only need one row per order.
# We'll take the LAST shipping_limit_date as the deadline for the whole order.
gaps_df_orders = gaps_merged.groupby('order_id').agg({
    'customer_unique_id': 'first',
    'order_purchase_timestamp': 'first',
    'order_approved_at': 'first',
    'order_delivered_carrier_date': 'first',
    'order_delivered_customer_date': 'first',
    'order_estimated_delivery_date': 'first',
    'shipping_limit_date': 'max' # Use the last deadline
}).reset_index()

# Calculate all gaps in hours (or days)
gaps_df_orders['avg_payment_approval_time'] = (gaps_df_orders['order_approved_at'] - gaps_df_orders['order_purchase_timestamp']).dt.total_seconds() / 3600
gaps_df_orders['avg_delivery_vs_estimate'] = (gaps_df_orders['order_estimated_delivery_date'] - gaps_df_orders['order_delivered_customer_date']).dt.total_seconds() / (24 * 3600)
gaps_df_orders['avg_carrier_shipping_time'] = (gaps_df_orders['order_delivered_customer_date'] - gaps_df_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)
gaps_df_orders['avg_shipping_vs_deadline'] = (gaps_df_orders['shipping_limit_date'] - gaps_df_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)

# Now, average these order-level gaps for each customer
gap_features = gaps_df_orders.groupby('customer_unique_id')[[
    'avg_payment_approval_time', 'avg_delivery_vs_estimate', 'avg_carrier_shipping_time',
    'avg_shipping_vs_deadline'
]].mean().reset_index()

# --- Merge this final group ---
dataframes["Customer Centric"] = pd.merge(dataframes["Customer Centric"], gap_features, on='customer_unique_id', how='left')

#### Feature Addition Post First Error Analysis - Purchase Intent
- Reasoning: Some product categories are likely to be purchased repeatedly, while others aren't
- Extraction: Find the repeat purchase rate of each product and get the max repeat purchase rate of products purchased by each customer

In [ ]:
print("--- Starting: Purchase Intent Feature Engineering ---")

# --- 1. Calculate Repeat Purchase Rates ---
print("Calculating repeat purchase rates...")
customer_category_purchases = products_merged.groupby(['customer_unique_id', 'product_category_name'])['order_id'].nunique().reset_index()
customer_category_purchases.rename(columns={'order_id': 'purchase_count'}, inplace=True)

total_buyers_per_category = customer_category_purchases.groupby('product_category_name')['customer_unique_id'].nunique()
repeat_buyers_per_category = customer_category_purchases[customer_category_purchases['purchase_count'] > 1].groupby('product_category_name')['customer_unique_id'].nunique()

category_stats_df = pd.DataFrame({
    'total_buyers': total_buyers_per_category,
    'repeat_buyers': repeat_buyers_per_category
})
category_stats_df['repeat_buyers'] = category_stats_df['repeat_buyers'].fillna(0).astype(int)
category_stats_df['repeat_purchase_rate'] = (category_stats_df['repeat_buyers'] / category_stats_df['total_buyers'])

# --- 2. Filter Out Categories Without Enough Data ---
MIN_BUYERS = 30 
category_stats_df = category_stats_df[category_stats_df['total_buyers'] >= MIN_BUYERS]

# --- 3. Define Baseline Rate and Calculate Purchase Intent Score ---
BASELINE_RATE = 0.01
category_stats_df['purchase_intent_score'] = category_stats_df['repeat_purchase_rate'] - BASELINE_RATE

# --- 4. Map Intent to Customers ---
category_score_map = category_stats_df['purchase_intent_score'].to_dict()
# Map this score to every purchase in our master dataframe
products_merged['purchase_intent_score'] = products_merged['product_category_name'].map(category_score_map)
# Aggregate by customer, taking the MAX score
customer_intent_df = products_merged.groupby('customer_unique_id')['purchase_intent_score'].max().reset_index()

# --- 5. Merge New Feature into the Main DataFrame ---
print("Merging 'purchase_intent_score' into dataframes['Customer Centric']...")

dataframes["Customer Centric"] = pd.merge(
    dataframes["Customer Centric"],
    customer_intent_df,
    on='customer_unique_id',
    how='left'
)

print("New continuous 'purchase_intent_score' feature added successfully.")
print(dataframes["Customer Centric"]['purchase_intent_score'].describe())

And we're done! Time to clean any NaNs.

In [ ]:
# Check for NaNs
print("NaNs before cleaning:")
print(dataframes["Customer Centric"].isnull().sum())

# Fill NaNs with logical defaults
dataframes["Customer Centric"]['avg_review_score'] = dataframes["Customer Centric"]['avg_review_score'].fillna(
    dataframes["Customer Centric"]['avg_review_score'].mean() # Impute with the mean score
)
dataframes["Customer Centric"]['review_engagement_rate'] = dataframes["Customer Centric"]['review_engagement_rate'].fillna(0) # 0% engagement
dataframes["Customer Centric"]['avg_payment_complexity'] = dataframes["Customer Centric"]['avg_payment_complexity'].fillna(1) # Default to 1

# Drop customer with no monetary value (never paid)
dataframes["Customer Centric"].dropna(subset=['Monetary'], inplace=True)

# Handle the 2-13 gap time NaNs
# Impute with the mean time for each
for col in ['avg_payment_approval_time', 'avg_delivery_vs_estimate', 
            'avg_carrier_shipping_time', 'avg_shipping_vs_deadline']:
    dataframes["Customer Centric"][col] = dataframes["Customer Centric"][col].fillna(
        dataframes["Customer Centric"][col].mean()
    )

# Fill final NaNs (customers with no categorized purchases) with the neutral score
dataframes["Customer Centric"]['purchase_intent_score'] = dataframes["Customer Centric"]['purchase_intent_score'].fillna(0.0)

print("NaNs after cleaning:")
print(dataframes["Customer Centric"].isnull().sum())

# Save the final file
dataframes["Customer Centric"].to_csv('customer_centric_features.csv', index=False)

print("--- Final Customer-Centric DataFrame Assembled ---")
print(dataframes["Customer Centric"].head())
print(dataframes["Customer Centric"].info())

Now we need to define what churn actually means and the feature dataframe needs to be preprocessed before using it to train the model. We'll start by defining churn as a customer having made no purchases within 180-days.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold

# --- 1. Define Target Variable (y) ---

# This is a key business decision. A 180-day window is a common starting point.
# "If you haven't bought in the last 6 months, you are considered churned."
CHURN_WINDOW_DAYS = 180

dataframes["Customer Centric"]['is_churned'] = (dataframes["Customer Centric"]['Recency'] > CHURN_WINDOW_DAYS).astype(int)
# (1 = Churned, 0 = Not Churned)

print(f"Churn window set to {CHURN_WINDOW_DAYS} days.")
print(dataframes["Customer Centric"]['is_churned'].value_counts(normalize=True))

We'll want to train the model on a set with the same amount of churners as the test set so we'll apply stratification. The data must be split prior to preprocessing so the StandardScaler does not learn the mean and standard deviation of the test set. It is also important to drop recency so the model does not just check if recency is over 180 and ignore the other features.

In [ ]:
# --- 2. Separate Features (X) and Target (y) ---
y = dataframes["Customer Centric"]['is_churned']
X = dataframes["Customer Centric"].drop(columns=['customer_unique_id', 'is_churned', 'Recency']) # Critical: Drop Recency to avoid target leakage

# --- 3. Split Data with Stratification ---
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

Numeric features need to be scaled within a normal range to avoid large values outcompeting smaller values. Preferred payment type will need to be one-hot encoded.

In [ ]:
# --- 4. Identify Numerical and Categorical Features ---
# All columns are numerical, except for 'preferred_payment_method'
numerical_features = X.columns.drop('preferred_payment_method')
categorical_features = ['preferred_payment_method']

print(f"\nIdentified {len(numerical_features)} numerical features.")
print(f"Identified {len(categorical_features)} categorical features.")

# --- 5. Create a Preprocessing Pipeline ---
# Step 1: Remove zero-variance features
# Step 2: Apply Quantile Transformation to stabilize variance
# Step 3: Scale numerical features
numeric_pipeline = Pipeline(steps=[
    ('variance_remover', VarianceThreshold(threshold=0.0)),
    ('quantile_transform', QuantileTransformer(output_distribution='normal')),
    ('scaler', StandardScaler())
])

# Create a OneHotEncoder for categorical features
# handle_unknown='ignore' prevents errors if new categories appear in test data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Use ColumnTransformer to apply transformers to the correct columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Keep any columns not listed (just in case)
)

# --- 6. Apply Preprocessing (The Correct Way) ---
# Fit and transform *only* the training data
print("\nFitting preprocessor on X_train...")
X_train_processed = preprocessor.fit_transform(X_train)

# --- DEBUGGING CHECKS ---
print("--- Checking for NaNs and Infs after preprocessing ---")

# Check for NaNs
nan_count = np.isnan(X_train_processed).sum()
print(f"Total NaNs created: {nan_count}")

# Check for Infs
inf_count = np.isinf(X_train_processed).sum()
print(f"Total Infs created: {inf_count}")

if nan_count > 0 or inf_count > 0:
    print("\nWARNING: Your preprocessing pipeline is creating bad values!")
else:
    print("\nNo NaNs or Infs found. The pipeline is clean.")
    # If clean, print the max values of each column
    print("\nMax value in each processed column:")
    print(X_train_processed.max(axis=0))

# *Only transform* the test data (using rules learned from train)
print("Transforming X_test...")
X_test_processed = preprocessor.transform(X_test)

print("Preprocessing complete.")
print(f"X_train_processed shape: {X_train_processed.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")

The training and test data is ready to go, time to train a few models! We'll start wtih logistic regression for a quick and easy baseline.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model_results = {}

# --- 1. Initialize and Train ---
print("--- Training: Logistic Regression ---")
lr_model = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000, solver='liblinear')
lr_model.fit(X_train_processed, y_train)

# --- 2. Get Predictions ---
y_pred_lr = lr_model.predict(X_test_processed)
y_prob_lr = lr_model.predict_proba(X_test_processed)[:, 1] # Get probabilities for the "churn" class

# --- 3. Evaluate ---
print("\n--- Logistic Regression Results ---")
print(classification_report(y_test, y_pred_lr, target_names=['Not Churned', 'Churned']))

roc_auc_lr = roc_auc_score(y_test, y_prob_lr)
print(f"ROC-AUC Score: {roc_auc_lr:.4f}")

model_results['Logistic Regression'] = {
    "ROC-AUC": roc_auc_lr,
    "Recall (Churn)": recall_score(y_test, y_pred_lr, pos_label=1),
    "Precision (Churn)": precision_score(y_test, y_pred_lr, pos_label=1)
}

Despite the strange and persistent numerical errors, the performance is not too bad. Now let's try a random forest model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("--- Training: RandomForestClassifier ---")

rf_model = RandomForestClassifier(
    class_weight='balanced', 
    random_state=42, 
    n_estimators=100, # A good default
    n_jobs=-1 # Use all available cores
)

rf_model.fit(X_train_processed, y_train)

# --- Evaluate ---
y_pred_rf = rf_model.predict(X_test_processed)
y_prob_rf = rf_model.predict_proba(X_test_processed)[:, 1]

print("\n--- RandomForestClassifier Results ---")
print(classification_report(y_test, y_pred_rf, target_names=['Not Churned', 'Churned']))
roc_auc_rf = roc_auc_score(y_test, y_prob_rf)
print(f"ROC-AUC Score: {roc_auc_rf:.4f}")

model_results['RandomForestClassifier'] = {
    "ROC-AUC": roc_auc_rf,
    "Recall (Churn)": recall_score(y_test, y_pred_rf, pos_label=1),
    "Precision (Churn)": precision_score(y_test, y_pred_rf, pos_label=1)
}

That is great performance! Let's try a gradient boosting model next.

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, recall_score, precision_score

print("--- Training: XGBoost Classifier ---")

# --- 1. Calculate scale_pos_weight for imbalance ---
# This is the XGBoost equivalent of class_weight='balanced'
# (Count of negative class) / (Count of positive class)
weight_ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Calculated scale_pos_weight: {weight_ratio:.2f}")


# --- 2. Initialize and Train ---
# 'n_jobs=-1' uses all your CPU cores
xgb_model = xgb.XGBClassifier(
    scale_pos_weight=weight_ratio,
    random_state=42,
    n_jobs=-1 
)

xgb_model.fit(X_train_processed, y_train)

# --- 3. Get Predictions ---
y_pred_xgb = xgb_model.predict(X_test_processed)
y_prob_xgb = xgb_model.predict_proba(X_test_processed)[:, 1]

# --- 4. Evaluate ---
print("\n--- XGBoost Classifier Results ---")
print(classification_report(y_test, y_pred_xgb, target_names=['Not Churned', 'Churned']))

roc_auc_xgb = roc_auc_score(y_test, y_prob_xgb)
print(f"ROC-AUC Score: {roc_auc_xgb:.4f}")

model_results['XGBoost'] = {
    "ROC-AUC": roc_auc_xgb,
    "Recall (Churn)": recall_score(y_test, y_pred_xgb, pos_label=1),
    "Precision (Churn)": precision_score(y_test, y_pred_xgb, pos_label=1)
}

Excellent score! The feature engineering and preprocessing pipeline have paid off. The next step is hyperparameter tuning. First, we'll use GridSearchCV to find the best combination of architecture hyperparameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

# --- Setup (assumes X_train_processed, y_train exist) ---
weight_ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Using scale_pos_weight: {weight_ratio:.2f}")

# --- STEP 1: Tune Architecture Hyperparameters ---
print("\n--- Starting Step 1: Tuning Model Architecture ---")

# Your 4x4x4x4x4 grid = 1024 combinations
param_grid_1 = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}

# Base estimator for the grid search
base_estimator = xgb.XGBClassifier(
    scale_pos_weight=weight_ratio, 
    random_state=42, 
    n_jobs=-1
)

# Setup the GridSearchCV
# cv=5 is standard, n_jobs=-1 uses all cores for the search
grid_search_1 = GridSearchCV(
    estimator=base_estimator,
    param_grid=param_grid_1,
    cv=5,
    scoring='roc_auc',
    verbose=2, # This will print updates as it runs
    n_jobs=-1
)


grid_search_1.fit(X_train_processed, y_train)

print(f"Best ROC-AUC from Step 1: {grid_search_1.best_score_:.4f}")
print("Best Architecture Parameters found:")
print(grid_search_1.best_params_)

# Store the best params for Step 2
step_1_best_params = grid_search_1.best_params_

Now we'll keep those architecure params and find the best combination of regularization params.

In [ ]:
# --- STEP 2: Tune Regularization Hyperparameters ---
print("\n--- Starting Step 2: Tuning Regularization ---")

# Define the new grid for regularization
param_grid_2 = {
    'gamma': [0, 0.1, 0.5, 1, 2],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.1, 0.5, 1, 2, 5]
}

# **CRITICAL:** Create a new estimator that *already has*
# the best parameters from Step 1.
# We use **step_1_best_params to "unpack" the dictionary
# of best settings we just found.
fine_tune_estimator = xgb.XGBClassifier(
    **step_1_best_params,  # Unpacks n_estimators, max_depth, etc.
    scale_pos_weight=weight_ratio, 
    random_state=42, 
    n_jobs=-1
)

# Setup the *second* GridSearchCV
grid_search_2 = GridSearchCV(
    estimator=fine_tune_estimator,
    param_grid=param_grid_2,
    cv=5,
    scoring='roc_auc',
    verbose=2,
    n_jobs=-1
)

grid_search_2.fit(X_train_processed, y_train)

print(f"Final Best ROC-AUC: {grid_search_2.best_score_:.4f}")
print("Final Best Regularization Parameters found:")
print(grid_search_2.best_params_)

# --- FINAL MODEL ---
# This is your fully optimized model
final_best_model = grid_search_2.best_estimator_

# You can now use this model to evaluate on your test set
y_pred_final = final_best_model.predict(X_test_processed)
y_prob_final = final_best_model.predict_proba(X_test_processed)[:, 1]

print("\n--- Final Tuned Model Results on Test Set ---")
print(classification_report(y_test, y_pred_final, target_names=['Not Churned', 'Churned']))
print(f"Final ROC-AUC Score: {roc_auc_score(y_test, y_prob_final):.4f}")

The final performace is fantasic! Let's save it so we don't have to run another grid search.

In [ ]:
import joblib

# 1. Save the final model
joblib.dump(final_best_model, 'final_churn_model.joblib')

# 2. Save the preprocessor (from cell 85)
joblib.dump(preprocessor, 'churn_preprocessor.joblib')

print("\nSuccessfully saved 'final_churn_model.joblib' and 'churn_preprocessor.joblib'.")

Now that the model is fine-tuned and powerful, we can run feature importance to establish which features are driving these accurate predictions.

In [ ]:
# --- 1. Get Feature Names ---
try:
    num_features = numerical_features.tolist() 
    cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
    all_feature_names = num_features + cat_features
    print(f"Retrieved {len(all_feature_names)} feature names.")
except Exception as e:
    print(f"Error: Make sure 'preprocessor', 'numerical_features', and 'categorical_features' are defined.")

# --- 2. Get Importances from your FINAL model ---
importances = final_best_model.feature_importances_ 

# --- 3. Create, Sort, and Print DataFrame ---
importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\n--- Final Model Feature Importances ---")
print(importance_df.to_string())

### Key Insights
 - The most impactful features are mostly logistics related rather than product related. This shows a need to focus on logistics optimization to find cheaper, faster, and more reliable shipping partners and improve delivery estimates. 
 
 - Payment method is also a huge predictor, revealing that debit card, boleto, and voucher shoppers are high-churn, though debit card and boleto are highest. This suggests a need to target debit card and boleto shoppers with discount offers immediately after their first purchase to build a habit.

 - Classic loyalty metrics (Monetary, Tenure, Frequency) are expectedly all high in the ranking.

 - Average review score and review engagement rate are surprisingly unimportant.

 - The new purchase_intent_score was somewhat helpful.

 ### High-Risk Profile
 - Based on this data a high-churn-risk customer is one who paid a high shipping fee for their order, used Boleto or a Debit Card to pay, and has only purchased once or twice.

### Error Analysis - What is throwing the model off?

In [ ]:
# --- 1. Get the ORIGINAL, UN-PROCESSED Test Data ---
# We use the index from y_test to select the original test rows from 'X'
X_test_original = X.loc[y_test.index]

# --- 2. Create the Analysis DataFrame ---
# Combine the original features, the true labels, and our model's predictions
error_analysis_df = X_test_original.copy()
error_analysis_df['actual_churn'] = y_test
error_analysis_df['predicted_churn'] = y_pred_final

# --- 3. Isolate the Mistakes ---

# False Negatives (FN): Model said 0, but it was 1. (THE "MISSED" ONES)
fn_df = error_analysis_df[
    (error_analysis_df['predicted_churn'] == 0) & 
    (error_analysis_df['actual_churn'] == 1)
]

# False Positives (FP): Model said 1, but it was 0. (THE "FALSE ALARMS")
fp_df = error_analysis_df[
    (error_analysis_df['predicted_churn'] == 1) & 
    (error_analysis_df['actual_churn'] == 0)
]

# --- 4. Get Statistical Summaries of the Mistakes ---

print(f"Total Test Samples: {len(error_analysis_df)}")
print(f"Total False Negatives (Missed Churn): {len(fn_df)}")
print(f"Total False Positives (False Alarms): {len(fp_df)}")

print("\n--- Profile of MISSED Churners (False Negatives) ---")
print(fn_df.describe())

print("\n--- Profile of LOYAL Customers We Flagged (False Positives) ---")
print(fp_df.describe())

### Interpretation of Analysis - First Model
These profiles are strikingly similar, suggesting there is a missing feature the model needs to consider. Maybe some product categories are more likely to be repeatedly purchased. It is worth exploring the dataset to see what categories are purchased repeatedly.

### Interpretation of Analysis - Second Model
The profiles are still about the same. The churn is likely based on factors outside of the data like customer service, website experience, returns, or involuntary churn.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("--- Starting: Visual Error Analysis ---")

# List of key features to compare
# These are your most important features, plus our 'happy churner' hypotheses
features_to_plot = [
    'avg_shipping_cost',       # Your #1 most important feature
    'Monetary',
    'avg_carrier_shipping_time',
    'Tenure',
    'avg_review_score',        # Key to the 'happy churner' theory
    'avg_delivery_vs_estimate',
    'purchase_intent_score'    # Our new feature
]

# --- Create the plots ---
# This creates a grid of plots, 2 columns wide
num_plots = len(features_to_plot)
num_cols = 2
num_rows = (num_plots + 1) // num_cols # (calculates rows needed)

plt.figure(figsize=(12, num_rows * 4)) # Make the figure tall enough

for i, feature in enumerate(features_to_plot):
    plt.subplot(num_rows, num_cols, i + 1) # Create a new subplot
    
    # Plot the distributions
    # 'kde' = Kernel Density Estimate (a smooth histogram)
    sns.kdeplot(fn_df[feature], label='Missed Churners (FN)', color='blue', fill=True)
    sns.kdeplot(fp_df[feature], label='False Alarms (FP)', color='red', fill=True)
    
    plt.title(f'Distribution of "{feature}" for Error Groups')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()

plt.tight_layout() # Prevents plots from overlapping
plt.savefig('error_analysis_distributions.png')

print("--- Visual Error Analysis Complete ---")
print("Saved plots to 'error_analysis_distributions.png'")

### Final Thoughts
The overlapping graphs in this final visual error analysis suggests that we've hit the point of diminishing returns. The features we have available are just not useful in telling these groups apart, there is some outside ambiguity. However, I'm incredibly satisfied with the models performance. A ROC-AUC of 0.9418 indicates a powerful, highly performant model so it's time to move on to the longitudinal model.